In [ ]:
ad = '#TENANT_ID#'
sp = '#APP_ID#'
se = '#SECRET_KEY#'


spark.conf.set("fs.azure.account.auth.type.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", sp)
spark.conf.set("fs.azure.account.oauth2.client.secret.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", se)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", "https://login.microsoftonline.com/" + ad + "/oauth2/token")

In [ ]:
import dlt
from pyspark.sql.functions import struct
from pyspark.sql.functions import col
from pyspark.sql.types import DateType
from pyspark.sql.types import *
import pyspark.sql.functions as f
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
import re
import itertools
import numpy as np
from delta.tables import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType, TimestampType, DecimalType
from pyspark.sql.functions import regexp_replace

In [ ]:
AQI_input_file = "abfss://data-source@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/Bed Occupancy and Availability Report/AQIData.csv"
from pyspark.sql.functions import col

@dlt.table()
def AQI_bronze():
  return (spark.read.format("csv").option("header",True).load(AQI_input_file))

@dlt.table()
def AQI_silver():
  return (dlt.read("AQI_bronze").select(col("msrDeviceNbr").alias("MsrDeviceNbr"),col("readingDateTimeUTC").alias("ReadingDateTimeUTC"),col("tempC").alias("TempCelcius"),col("humidity").alias("Humidity"),col("pressure").alias("Pressure"),col("pM25").alias("PM25"),col("pM10").alias("PM10"),col("pM1").alias("PM1"),col("aqi").alias("AQI"),col("merge").alias("Merge"),col("hour").alias("Hour"),col("Date").alias("Date"),col("Time").alias("Time"),col("Column").alias("Commute"),col("LocationKey").alias("LocationKey"),col("TimeCopy").alias("TimeInHrs"),col("Month1").alias("MonthName"),col("Year").alias("Year"),col("week").alias("WeekNumber"),col("aqiLabel").alias("AQILabel"),col("AQILabelID").alias("AQILabelID"),col("latitude1").alias("latitude"),col("longitude1").alias("longitude"),col("City").alias("City")))

@dlt.table()
def AQI_gold():
  return (dlt.read("AQI_silver"))

In [ ]:
# Supplier_input_file = "abfss://data-source@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/Bed Occupancy and Availability Report/Healthdata.csv"
# df=spark.read.format("csv").option("header",True).load(Supplier_input_file)
# df.show()


In [ ]:

Suppliers_input_file = "abfss://data-source@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/Bed Occupancy and Availability Report/SupplierData.csv"

@dlt.table()
def Supplier_bronze():
  return (spark.read.format("csv").option("header",True).load(Suppliers_input_file))

@dlt.table()
def Supplier_silver():
  return (dlt.read("Supplier_bronze"))

@dlt.table()
def Supplier_gold():
  return (dlt.read("Supplier_silver"))


In [ ]:

OccupancyHealthcareData = "abfss://data-source@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/Bed Occupancy and Availability Report/OccupancyHealthcare_Data.csv"
@dlt.table()
def OccupancyHealthcareData_bronze():
  return (spark.read.format("csv").option("header",True).load(OccupancyHealthcareData))

@dlt.table()
def OccupancyHealthcareData_silver():
  return (dlt.read("OccupancyHealthcareData_bronze").select(col("Date").alias("Date"),col("City").alias("City"),col("Region").alias("Region"),col("BedOccupancyPercentage").alias("BedOccupancyPercentage"),col("AQIIndex").alias("AQI_Index"),col("Latitude").alias("Latitude"),col("Longitude").alias("Longitude"),col("CountofID").alias("CountOfID"),col("Bysymptom").alias("BySmptom"),col("ShippingDelays").alias("ShippingDelay"),col("AvailableCencus").alias("AvailableCencus"),col("MaxBed").alias("MaxBed")))

@dlt.table()
def OccupancyHealthcareData_gold():
  return (dlt.read("OccupancyHealthcareData_silver"))

#Bed Occupancy Powered by AQI And Suppiler

In [ ]:
@dlt.table(
    comment="Bed Occupancy Powered by AQI And Suppiler ",
    table_properties={
    "quality": "gold"
    }    
) 

def AirIndex_Analytics():
    return (
        spark.sql("""
         select a.* from live.OccupancyHealthcareData_gold a,
              live.Supplier_gold b,live.AQI_gold c
   where a.City=b.City or c.City=a.City
        """)
    )